# Library

Test for a library that is able to run argo pipelines created via python

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
    
import rmipipeline

In [3]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

In [4]:
coreV1Api = client.CoreV1Api()

In [5]:
unique_name = rmipipeline.create_local_storage(coreV1Api, 
                     "0040013", 
                     "default", 
                     "/home/michele/Projects/rmi-pipelines/data/recon_all/controllo/0040013", 
                     "10Mi", 
                     "ReadOnlyMany")

In [6]:
print(unique_name)

0040013-95dceb0a


In [5]:
rmipipeline.delete_all_pv_and_pvc(coreV1Api, "0040013", "default")

Deleted 0 PersistentVolumes and 0 PersistentVolumeClaims


In [6]:
from argo.workflows.client.models import *
from argo.workflows.client import V1alpha1Api
from argo.workflows.config import load_kube_config

In [7]:
config.load_kube_config()

In [8]:
"""
create_config_map_with_license_file create a ConfigMap in a namespace given the license file.
This config map will be used to run FreeSurfer processing.
"""
def create_config_map_with_license_file(path="license.txt", namespace="default"):
    
    # Read license file
    f = open(path, "r")
    license = f.read()
    f.close()
    
    # Create a config map with the FreeSurfer license
    coreV1Api.create_namespaced_config_map(
        body=V1ConfigMap(
            metadata=V1ObjectMeta(
                name="freesurfer-license",
                namespace=namespace    
            ),
            data={
                "license.txt": license
            }
        ), 
        namespace="default"
    )    

In [9]:
create_config_map_with_license_file("license.txt")

ApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Fri, 10 Jan 2020 11:14:42 GMT', 'Content-Length': '226'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"configmaps \"freesurfer-license\" already exists","reason":"AlreadyExists","details":{"name":"freesurfer-license","kind":"configmaps"},"code":409}



In [10]:
p = rmipipeline.Pipeline()

In [11]:
p.exposeClaimWithName(unique_name)

NameError: name 'unique_name' is not defined

In [15]:
task = rmipipeline.FreeSurfer("T4", "mri_surf2surf --hemi lh --tval lh.pippo.aparc.annot --sval-annot /opt/freesurfer/subjects/fsaverage/label/lh.aparc.a2005s.annot --srcsubject fsaverage --trgsubject 0040013")

task.mountVolumeAt(unique_name, "/opt/freesurfer/subjects/0040013")

In [16]:
t4 = p.add(task)

In [17]:
p.run()

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 1, 6, 14, 20, 29, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': 'dag-diamond-',
              'generation': 1,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'dag-diamond-25nkb',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '1829583',
              'self_link': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/dag-diamond-25nkb',
              'uid': '132d5133-5fa9-4bd1-ba46-3599918f7371'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'arguments': None,
          'artifact_repository

In [22]:
%autoreload 2
    
import rmipipeline

In [14]:
%autoreload 2

from rmipipeline import Pipeline, Bash, FreeSurfer, Task

# load_kube_config()  # loads local configuration from ~/.kube/config
# v1alpha1 = V1alpha1Api()
# return v1alpha1.create_namespaced_workflow(self.namespace, workflow)

p = Pipeline()

t1 = p.add(Bash("T1", ["bash", "-c", "cd / ; ls -lh /opt/freesurfer ; cat /opt/freesurfer/license.txt ; echo michele > /workdir/test.txt ; ls -lh"], []))
t2 = p.add(Bash("T2", ["ls"], ["/"]))

# t2 = Task("T2", "alpine:3.7")
t3 = p.add(Task("T3", "alpine:3.7"))
# t4 = p.add(FreeSurfer("T4", "mri_surf2surf --hemi lh --tval lh.pippo.aparc.annot --sval-annot /opt/freesurfer/subjects/fsaverage/label/lh.aparc.a2005s.annot --srcsubject fsaverage --trgsubject 0040013"))
t4 = p.add(Task("T4", "alpine:3.7"))

"""
!docker run \
    --user "$(id -u $USER):$(id -g $USER)" \
    -v $(pwd)/license.txt:/opt/freesurfer/license.txt \
    -v $(pwd)/data/recon_all/controllo/0040013:/opt/freesurfer/subjects/0040013 \
        freesurfer/freesurfer:6.0 \
            mri_surf2surf \
                --hemi lh \
                --tval lh.pippo.aparc.annot \
                --sval-annot /opt/freesurfer/subjects/fsaverage/label/lh.aparc.a2005s.annot \
                --srcsubject fsaverage \
                --trgsubject 0040013    
"""

t5 = p.add(Task("T5", "alpine:3.7"))
t6 = p.add(Task("T6", "alpine:3.7"))
t7 = p.add(Task("T7", "alpine:3.7"))

t1.set_upstream(t2).set_upstream(t3)
t5.set_upstream(t4).set_upstream(t2)
t6.set_upstream(t4)
t6.set_upstream(t1)
t7.set_upstream(t5)
t7.set_upstream(t1)

p.run()

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 1, 10, 11, 16, 14, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': 'dag-diamond-',
              'generation': 1,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'dag-diamond-46g9p',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '2084188',
              'self_link': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/dag-diamond-46g9p',
              'uid': '32fac476-e627-4e9f-bb9e-3bfea3aec2bb'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'arguments': None,
          'artifact_repositor